In [1]:
import openpyxl as op
import xlsxwriter 
import sympy as sy 

In [12]:

def matricial(number_of_elements, filename, number_of_nodes, sheet__name, cant_springs):
    """
    Función que calcula la solución del método matricial

    :param number_of_elements: número de elementos del sistema
    :param filename: nombre del archivo de Excel con los datos
    :param number_of_nodes: número de nodos del sistema
    :param sheet_name: nombre de la hoja de Excel con los datos
    :param cant_springs: cantidad de resortes del sistema
    :return: None
    """

    degrees_freedom = 3*number_of_nodes

    excel_document = op.load_workbook(filename)
    data_excel = excel_document[sheet__name]

    k_global = k_assembled(degrees_freedom, number_of_elements, data_excel) #se ensambla la matriz de rigidez global

    #se itera y lee el valor sobre cada columna y fila que contiene los valores de las fuerzas de empotramiento en el excel
    fep = sy.Matrix([[data_excel.cell(row=2+i, column=12+j).value] for i in range(number_of_nodes) for j in range(3)]) #fuerzas de empotramiento perfecto/perfect embedment forces

    #se lee los desplazamientos de las filas y columnas del excel, se tranforma a valores simbolicos lo que no sea float o int
    displacements = [data_excel.cell(row=2+i, column=15+j).value for i in range(number_of_nodes) for j in range(3)]
    displacements = sy.Matrix(list(map(lambda x: sy.symbols(x) if not isinstance(x, (float, int)) else x, displacements)))

    #se lee las reacciones de las filas y columnas del excel, se tranforma a valores simbolicos lo que no sea float o int
    reactions = [data_excel.cell(row=2+i, column=18+j).value for i in range(number_of_nodes) for j in range(3)]
    reactions = sy.Matrix(list(map(lambda x: sy.symbols(x) if not isinstance(x, (float, int)) else x, reactions)))


    support_rotation = matrix_support_rotation(data_excel, number_of_nodes) #angulo de inclinacion de apoyos
    k_springs = springs(data_excel, number_of_nodes, cant_springs)#vector con la rigidez de cada resorte en el sistema

    #se aplica la rotacion a las fuerzas de empotramiento perfecto dependiendo del angulo de rotacion de cada apoyo respecto a plano de referencia global
    fep = support_rotation*fep

    #se aplica rotacion a la matriz global ensablada dependiendo del angulo de rotacion de cada apoyo respecto al plano de referencia global
    k_global = support_rotation*k_global*support_rotation.T
    k_global = k_global+k_springs
     
    unknowns = [] #incognitas de los sistemas de ecuaciones a resolver, corresponde a los desplazamientos y reacciones
    #se evalua si se aplicara condensacion estatica

    #ciclo para colocar las variables simbolicas de "reactions" y "displacements" en el vector "unknows"
    for i in range(0, degrees_freedom):
    #se evalua para cada grado de libertad si el valor del vector "reactions" es un simbolo de ser asi se agrega a "unknows", en caso contrario se agrega el valor "displacemnet"
        if type(reactions[i, 0]) == sy.core.symbol.Symbol:
            unknowns += [reactions[i, 0]]
        else:
            unknowns += [displacements[i, 0]]   

    system_equations = k_global*displacements - reactions + fep #se opera la ecuacion general
    solve_system_equations = sy.solve(system_equations,unknowns) #se soluciona el sistema de ecuaciones

    print(solve_system_equations) #se imprime solucion del sistema de ecuaciones
    Guardar(k_global, "KTotal.xlsx") #se guarda la matriz de rigidez global ensamblada en una hoja de excel

#funcion para crear y ensamblar la matriz de rigidez total
def k_assembled(degrees_freedom, number_element, data_excel):

    KT = sy.Matrix.zeros(degrees_freedom,degrees_freedom)
    
    #el ciclo itera para cada final de datos como elementos haya, y crea la matriz local de cada elemento, se tranforma y agrega a la matriz global
    for i in range(number_element): #se itera sobre cada fila como elementos haya

        #se almacena las primeras 11 columnas del excel para la i fila, corresponden a las propiedades del elemento i
        data_element = [data_excel.cell(row=2+i, column=1+j).value for j in range(11)]

        A = data_element[1] #Area
        E = data_element[2] #Modulo de elasticidad
        I = data_element[3] #momento de inercia
        L = data_element[4] #longitud
        theta = data_element[5]*sy.pi/180 #angulo de inclinacion de ejes locales respecto al plano global
        Nx, Ny, Nz = range(data_element[6]*3-3,data_element[6]*3) #grados de libertad inicial
        Fx, Fy, Fz = range(data_element[7]*3-3,data_element[7]*3) #grados de libertad final
        y = round(sy.sin(theta),2) #angulo en y
        x = round(sy.cos(theta),2) #angulo en x
        Ai = data_element[8] #condicion de apoyo incial
        Aj = data_element[9] #condicion de apoyo final
        b = data_element[10] #factor de beta de vigas timoshenko

        #matriz de tranfromacion de ejes globales a locales
        m_tranformacion = sy.Matrix([[x, -y, 0, 0,  0, 0], 
                                     [y,  x, 0, 0,  0, 0],
                                     [0,  0, 1, 0,  0, 0],
                                     [0,  0, 0, x, -y, 0],
                                     [0,  0, 0, y,  x, 0],
                                     [0,  0, 0, 0,  0, 1]])
 
        k_l = sy.Matrix.zeros(6,6) #matriz loca en terminos globales al ser elementos 2d siempre sera de 6x6

        #se evalua cada condicion de apoyo/union de cada elementos, se opera la matriz de rigidez dependiendo de la condicion de apoyos 
        #A: union con liberacion de momentos (articulacion/pinned)
        #R: unicon rigida (tranferencia de momento, cortante, axial)
        #V: union con liberacion de cortante (tranferencia de momento, axial)
        #X: union con liberacion de axial (tranfereancia de momento, cortante)

        if Ai == 'A' and Aj == 'A':
            k_l = sy.Matrix([[A*(L**2)*(1+b),  0, 0, -A*(L**2)*(1+b), 0, 0], 
                             [0,               0, 0,               0, 0, 0],
                             [0,               0, 0,               0, 0, 0],
                             [-A*(L**2)*(1+b), 0, 0,  A*(L**2)*(1+b), 0, 0],
                             [0,               0, 0,               0, 0, 0],
                             [0,               0, 0,               0, 0, 0]])*(E/((1+b)*L**3))
        if Ai == 'R' and Aj == 'R':
            k_l = sy.Matrix([
                [A*(L**2)*(1+b)/I,       0,               0,    -A*(L**2)*(1+b)/I,     0,               0], 
                [0,                     12,             6*L,                    0,   -12,             6*L],
                [0,                    6*L,    (L**2)*(4+b),                    0,  -6*L,    (L**2)*(2-b)],
                [-A*(L**2)*(1+b)/I,      0,               0,     A*(L**2)*(1+b)/I,     0,               0],
                [0,                    -12,            -6*L,                    0,    12,            -6*L],
                [0,                    6*L,    (L**2)*(2-b),                    0,  -6*L,    (L**2)*(4+b)]
                ])*(E*I/((1+b)*L**3))
        if Ai == 'R' and Aj == 'A':
            k_l = sy.Matrix([
                [A*(L**2)*(1+b)/I,                    0,                                                        0,    -A*(L**2)*(1+b)/I,                    0,  0], 
                [0,                        12-(36/(b+4)),                                       12*L-(36*L/(b+4)),                    0,        (36/(b+4))-12,  0],
                [0,                    12*L-(36*L/(b+4)),   (-36*(L**2)/(b+4))+(b+4)*(L**2)-(b*(L**2)) + 8*(L**2),                    0,    (36*L/(b+4))-12*L,  0],
                [-A*(L**2)*(1+b)/I,                    0,                                                       0,     A*(L**2)*(1+b)/I,                    0,  0],
                [0,                        (36/(b+4))-12,                                       (36*L/(b+4))-12*L,                    0,        12-(36/(b+4)),  0],
                [0,                                    0,                                                       0,                    0,                    0,  0]
                ])*(E*I/((1+b)*L**3))
        if Ai == 'A' and Aj == 'R':
            k_l = sy.Matrix([
                [A*(L**2)*(1+b)/I,                     0,   0,  -A*(L**2)*(1+b)/I,                  0,                                                      0], 
                [0,                        12-(36/(b+4)),   0,                  0,      (36/(b+4))-12,                                      12*L-(36*L/(b+4))],
                [0,                                    0,   0,                  0,                  0,                                                      0],
                [-A*(L**2)*(1+b)/I,                    0,   0,   A*(L**2)*(1+b)/I,                  0,                                                      0],
                [0,                        (36/(b+4))-12,   0,                  0,      12-(36/(b+4)),                                      (36*L/(b+4))-12*L],
                [0,                    12*L-(36*L/(b+4)),   0,                  0,  (36*L/(b+4))-12*L,  (-36*(L**2)/(b+4))+(b+4)*(L**2)-(b*(L**2)) + 8*(L**2)]
                ])*(E*I/((1+b)*L**3))
        if Aj == 'V' or Ai == 'V':
            k_l = sy.Matrix([[A*(L**2)*(1+b)/I,  0,             0, -A*(L**2)*(1+b)/I, 0,             0], 
                             [0,                 0,             0,                 0, 0,             0],
                             [0,                 0,  (b+1)*(L**2),                 0, 0, -(b+1)*(L**2)],
                             [-A*(L**2)*(1+b)/I, 0,             0,  A*(L**2)*(1+b)/I, 0,             0],
                             [0,                 0,             0,                 0, 0,             0],
                             [0,                 0, -(b+1)*(L**2),                 0, 0,  (b+1)*(L**2)]
                             ])*round((E*I / ((1+b)*L**3)),8)
        if Ai == 'X' or Aj == 'X':
            k_l = sy.Matrix([[0,       0,             0,    0,      0,            0], 
                             [0,    12*L,           6*L,    0,  -12*L,          6*L],
                             [0,     6*L,  (b+4)*(L**2),    0,   -6*L, (L**2)*(2-b)],
                             [0,       0,             0,    0,      0,            0],
                             [0,   -12*L,          -6*L,    0,   12*L,         -6*L],
                             [0,     6*L,  (L**2)*(2-b),    0,   -6*L, (L**2)*(4+b)]
                             ])*(E*I/((1+b)*L**3))

        #se tranforma la matriz local a ejes globales
        k_l = m_tranformacion*k_l*m_tranformacion.T

        k_local = sy.Matrix.zeros(degrees_freedom,degrees_freedom)
        k_local[Nx,Nx] = k_l[0,0]
        k_local[Ny,Nx] = k_local[Nx,Ny] = k_l[0,1]
        k_local[Nz,Nx] = k_local[Nx,Nz] = k_l[0,2]
        k_local[Fx,Nx] = k_local[Nx,Fx] = k_l[0,3]
        k_local[Fy,Nx] = k_local[Nx,Fy] = k_l[0,4]
        k_local[Fz,Nx] = k_local[Nx,Fz] = k_l[0,5]
        k_local[Ny,Ny] = k_l[1,1]
        k_local[Nz,Ny] = k_local[Ny,Nz] = k_l[1,2]
        k_local[Fx,Ny] = k_local[Ny,Fx] = k_l[1,3]
        k_local[Fy,Ny] = k_local[Ny,Fy] = k_l[1,4]
        k_local[Fz,Ny] = k_local[Ny,Fz] = k_l[1,5]
        k_local[Nz,Nz] = k_l[2,2]
        k_local[Fx,Nz] = k_local[Nz,Fx] = k_l[2,3]
        k_local[Fy,Nz] = k_local[Nz,Fy] = k_l[2,4]
        k_local[Fz,Nz] = k_local[Nz,Fz] = k_l[2,5]
        k_local[Fx,Fx] = k_l[3,3]
        k_local[Fy,Fx] = k_local[Fx,Fy] = k_l[3,4]
        k_local[Fz,Fx] = k_local[Fx,Fz] = k_l[3,5]
        k_local[Fy,Fy] = k_l[4,4]
        k_local[Fz,Fy] = k_local[Fy,Fz] = k_l[4,5]
        k_local[Fz,Fz] = k_l[5,5]

        #adhiere cada matriz local en terminos globales a la matriz total
        KT = KT + k_local 

    return KT

#funcion para ensamblar la matriz con angulos para tranformacion de la matriz de rigidez dado la rotacion de los apoyos
def matrix_support_rotation(data_excel, number_of_nodes):
    g_nodos = sy.Matrix.zeros(3*number_of_nodes, 3*number_of_nodes)
    #ciclo para iterar la cantidad de nodos que tenga la estructrua, lee el el angulo de inclinacion de la columna 21 del excel
    for i in range(number_of_nodes):
        nodo = data_excel.cell(row=2+i, column=21).value
        theta = nodo*sy.pi/180
        Nx, Ny, Nz = range((i+1)*3-3,(i+1)*3) #grados de libertad inicial
        y = round(sy.sin(theta),8) #angulo en y
        x = round(sy.cos(theta),8) #angulo en x
        g_nodos[Nx,Nx] = x
        g_nodos[Nx,Ny] = y
        g_nodos[Ny,Nx] = -y
        g_nodos[Ny,Ny] = x
        g_nodos[Nz,Nz] = 1

    return g_nodos

#funcion para ensamblar la matriz con los valores de los resosrtes que tenga el sistema
def springs(data_excel, number_of_nodes,cant_springs):
    K_resortes = sy.Matrix.zeros(3*number_of_nodes,3*number_of_nodes)
    for i in range(cant_springs):
        gl_r = data_excel.cell(row=2+i, column=22).value
        k_r = data_excel.cell(row=2+i, column=23).value
        K_resortes[gl_r,gl_r] = k_r
    return K_resortes

#funcion para guardar matrices en excel
def Guardar(KT, name):
    workbook = xlsxwriter.Workbook(name) 
    worksheet = workbook.add_worksheet()
    row = 0
    for col, data in enumerate(KT.tolist()):
        worksheet.write_column(row, col, data)
    workbook.close()


In [13]:
#Ejemplo

#Numero total de elementos en la estructura
number_of_elements = 1
#numero de nodos
number_of_nodes = 2
#Nombre del archivo de excel con los datos
filename = "Q.xlsx"
#nombre de la hoja con los datos
sheet__name = "Hoja2"
#cantidad de resortes de la estrucutura
cant_springs = 0


matricial(number_of_elements, filename, number_of_nodes, sheet__name, cant_springs)

{FX1: 0.0, FY1: 3561.40056554896, M1: 3409.58833666932, FX2: 0.0, FY2: -2586.40056554896, M2: 6599.61335997755}
